In [10]:
import csv
with open('enjoySport.csv') as csvFile:
  examples = [tuple(line)[1:] for line in csv.reader(csvFile)]

examples = examples[1:]
examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Y'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Y'),
 ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'N'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Y')]

In [11]:
def s_0(n):
  return ('0',)*n

def g_0(n):
  return ('?', )*n

In [19]:
def get_domains(examples):
  d = [set() for x in examples[0]]
  for x in examples:
    for ind, xi in enumerate(x):
      d[ind].add(xi)
  
  return [list(sorted(x)) for x in d]
get_domains(examples)

[['Rainy', 'Sunny'],
 ['Cold', 'Warm'],
 ['High', 'Normal'],
 ['Strong'],
 ['Cool', 'Warm'],
 ['Change', 'Same'],
 ['N', 'Y']]

In [13]:
def more_general(h1, h2):
  mg_flags = list()
  for h,x in zip(h1, h2):
    flag = h == '?' or (h!='0' and (h==x or x=='0'))
    mg_flags.append(flag)
  
  return all(mg_flags)

In [14]:
def min_specialisation(hg, x, domains):
  result = []
  for i in range(len(hg)):
    if hg[i] == '?':
      for val in domains[i]:
        if val != x[i]:
          hg_new = hg[:i] + (val, ) + hg[i+1:]
          result.append(hg_new)
    
    if hg[i] != '0':
      hg_new = hg[:i] + ('0', ) + hg[i+1:]
      result.append(hg_new)
  
  return result

In [15]:
def min_generalisation(hs, x):
  hs_new = list(hs)
  for i in range(len(hs)):
    if not more_general(hs[i:i+1], x[i:i+1]):
      hs_new[i] = '?' if hs[i] != '0' else x[i]
  return [tuple(hs_new)]

In [16]:
def specialise_G(S, G, x, domains):
  G_prev = list(G)
  for g in G_prev:
    if g not in G:
      continue
    if more_general(g, x):
      G.remove(g)
      G_minus = min_specialisation(g, x, domains)
      G.update([h for h in G_minus if any([more_general(h, s) for s in S])])
      G.difference_update([h for h in G if any([more_general(h1, h) for h1 in G if h != h1])])
  
  return G


In [17]:
def generalise_S(S, G, x):
  S_prev = list(S)

  for s in S_prev:
    if s not in S:
      continue
    if not more_general(s, x):
      S.remove(s)
      Splus = min_generalisation(s, x)

      S.update([h for h in Splus if any(more_general(g, h) for g in G)])
      S.difference_update([h for h in S if any(more_general(h1, h) for h1 in S if h!=h1)])
  
  return S


In [18]:
domains = get_domains(examples)[:-1]
print(domains)
S = set([s_0(len(domains))])
G = set([g_0(len(domains))])
i = 0

print(f'S[{i}]->{S}')
print(f'G[{i}]->{G}')

for ex in examples:
  i+=1
  ex = list(ex)
  x, label = ex[:-1], ex[-1]
  print(label)
  if label == 'Y':
    G = {g for g in G if more_general(g, x)}
    S = generalise_S(S, G, x)
  elif label == 'N':
    S = {s for s in S if not more_general(s, x)}
    G = specialise_G(S, G, x, domains)
  
  print(f'S[{i}]->{S}')
  print(f'G[{i}]->{G}')
  

[['Rainy', 'Sunny'], ['Cold', 'Warm'], ['High', 'Normal'], ['Strong'], ['Cool', 'Warm'], ['Change', 'Same']]
S[0]->{('0', '0', '0', '0', '0', '0')}
G[0]->{('?', '?', '?', '?', '?', '?')}
Y
S[1]->{('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}
G[1]->{('?', '?', '?', '?', '?', '?')}
Y
S[2]->{('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
G[2]->{('?', '?', '?', '?', '?', '?')}
N
S[3]->{('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}
G[3]->{('Sunny', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same'), ('?', 'Warm', '?', '?', '?', '?')}
Y
S[4]->{('Sunny', 'Warm', '?', 'Strong', '?', '?')}
G[4]->{('Sunny', '?', '?', '?', '?', '?'), ('?', 'Warm', '?', '?', '?', '?')}
